In [1]:
import pandas as pd

# Eksploracja danych nypd-motor-vehicle-collisions.csv

## Wczytanie danch
Wczytanie danych o wypadkach i kolizjach w mieście Nowy Jork z pliku csv.
Plik posiada nagłówek z nazwami kolumn a jako separatora użyto znaku ','

In [2]:
# Wczytuję dana z pliku csv
ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv")

/tmp/ipykernel_5381/3714160385.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv")


In [4]:
ny_collisions.head(3)

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225.0,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002.0,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN


In [5]:
# Z powodu komunikatu Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
# przyglądam się kolumnie (3) "ZIP CODE"
ny_collisions["ZIP CODE"]

0          11434.0
1          11225.0
2          10002.0
3          10460.0
4          10016.0
            ...   
1612173    11226.0
1612174        NaN
1612175        NaN
1612176    10024.0
1612177    11361.0
Name: ZIP CODE, Length: 1612178, dtype: object

Pandas potraktował wartości kolumny ZIP CODE jako liczby. Bezpośrednio w pliku CSV jest to głównie 5 cyfr, ponadto kolumna posiada wartości puste lub wypełnione białymi znakami.
Traktuję kolumnę **ZIP CODE** jako ciąg znaków (typ str) a kolumnę **ACCIDENT DATE** jako dane typu datetime i wczytuję dane ponownie.

In [6]:
# kolumna ZIP CODE jako str, a ACCIDENT DATE jako datetime
ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv",dtype={"ZIP CODE":'str'}, parse_dates=["ACCIDENT DATE"])
ny_collisions.head(3)

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05,16:30,QUEENS,11434,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27,16:02,BROOKLYN,11225,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15,17:57,MANHATTAN,10002,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN


## Podstawowe informacje o wczytanym DataFrame
Wczytany DataFrame ny_collisions posiada 1 612 178 wierszy w 29 kolumnach.

Dane o kolumnach DataFrame ny_collisions:
* **ACCIDENT DATE**: Data wypadku, podczas wczytywania kolumna przekształcona na typ danych 'datetime64[ns]'
* **ZIP CODE**: Kod pocztowy, typ danych ciąg znaków 'str'

In [8]:
# Podstawowe informacje o DataFrame ny_colision
# 1612178 wierszy i 29 kolumn
ny_collisions.shape

(1612178, 29)

In [10]:
ny_collisions.describe

<bound method NDFrame.describe of         ACCIDENT DATE ACCIDENT TIME    BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
0          2019-08-05         16:30     QUEENS    11434  40.676052 -73.790184   
1          2019-08-27         16:02   BROOKLYN    11225  40.657780 -73.951096   
2          2019-08-15         17:57  MANHATTAN    10002  40.718143 -73.993835   
3          2019-08-30         21:53      BRONX    10460  40.840534 -73.866610   
4          2019-08-06          9:45  MANHATTAN    10016  40.745440 -73.975400   
...               ...           ...        ...      ...        ...        ...   
1612173    2012-07-21         22:46   BROOKLYN    11226  40.651741 -73.965241   
1612174    2012-07-09         16:45        NaN      NaN        NaN        NaN   
1612175    2012-07-09         14:35        NaN      NaN  40.791492 -73.935763   
1612176    2012-07-18          9:20  MANHATTAN    10024  40.779479 -73.973573   
1612177    2012-07-12          3:10     QUEENS    11361  40.762501 -73.7825

In [11]:
ny_collisions.info

<bound method DataFrame.info of         ACCIDENT DATE ACCIDENT TIME    BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
0          2019-08-05         16:30     QUEENS    11434  40.676052 -73.790184   
1          2019-08-27         16:02   BROOKLYN    11225  40.657780 -73.951096   
2          2019-08-15         17:57  MANHATTAN    10002  40.718143 -73.993835   
3          2019-08-30         21:53      BRONX    10460  40.840534 -73.866610   
4          2019-08-06          9:45  MANHATTAN    10016  40.745440 -73.975400   
...               ...           ...        ...      ...        ...        ...   
1612173    2012-07-21         22:46   BROOKLYN    11226  40.651741 -73.965241   
1612174    2012-07-09         16:45        NaN      NaN        NaN        NaN   
1612175    2012-07-09         14:35        NaN      NaN  40.791492 -73.935763   
1612176    2012-07-18          9:20  MANHATTAN    10024  40.779479 -73.973573   
1612177    2012-07-12          3:10     QUEENS    11361  40.762501 -73.782522

In [13]:
# Nazwy kolumn DataFrame ny_colision
ny_collisions.columns

Index(['ACCIDENT DATE', 'ACCIDENT TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [15]:
# Typy danych kolumn
print("ACCIDENT DATE: ",ny_collisions["ACCIDENT DATE"].dtype)
print("ZIP CODE: ",ny_collisions["ZIP CODE"].dtype)

ACCIDENT DATE:  datetime64[ns]
ZIP CODE:  object
